# Двухфакторный дисперсионный анализ (Two-Way ANOVA)
Данный метод анализа данных применяется в тех случаях, когда мы хотим узнать, как различные свойства данных влияют на данные в целом. В частности, как два свойства влияют друг на друга.
Для того, чтобы провести двухфакторный дисперсионный анализ, необходимо ввести следующие понятия:
- SST (Sum square total) - Общая сумма квадратов;
- SSB_a (Sum square between) - Межгрупповая сумма квадратов групп по свойству А;
- SSB_b (Sum square between) - Межгрупповая сумма квадратов групп по свойству B;
- SSb_a*b - Межгрупповая сумма квадратов взаимодействия различных групп;
- SSW (Sum square within) - Внутригрупповая сумма квадратов.

Тогда:
SST = SSW + SSB_a + SSB_b + SSB_a*b
SSB_a*b можно найти, зная все остальные слагаемые и общую сумму квадратов.

Ниже по шагам вручную будет представлен алгоритм применения двухфакторного дисперсионного анализа.
Также предлагаю плейлист коротких видео с объяснением двухфакторного анализа - его рассчетами и интерпретацией результатов: 

## Инициализация.
Первым делом импортируем все необходимые библиотеки и создадим анализируемый датасет. Будем применять алгоритм на примере анализа данных об успеваемости студентов в зависимости от их пола и факультета.

In [18]:
import math
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import pandas as pd

df = pd.DataFrame(
    {
        "Sex": ["Male", "Male", "Male", "Female", "Female", "Female", "Male", "Male", "Male", "Female", "Female", "Female", "Male", "Male", "Male", "Female", "Female", "Female"],
        "Department": ["Math", "Math", "Math", "Math", "Math", "Math", "Biology", "Biology", "Biology", "Biology", "Biology", "Biology", "Politics", "Politics", "Politics", "Politics", "Politics", "Politics"],
        "Score": [9, 8, 9, 5, 4, 6, 6, 5, 6, 10, 9, 10, 5, 7, 6, 6, 7, 7]  
    }
)

df

,Sex,Department,Score
0,Male,Math,9
1,Male,Math,8
2,Male,Math,9
3,Female,Math,5
4,Female,Math,4
5,Female,Math,6
6,Male,Biology,6
7,Male,Biology,5
8,Male,Biology,6
9,Female,Biology,10


## Считаем средние.
Теперь посчитаем все необходимые средние значения для всех возможных групп.

In [19]:
# Общее среднее
overall_average = np.average(df["Score"])                                     

# Средние для групп по свойству "Пол"
male_average   = np.average(df.loc[df["Sex"] == "Male"]["Score"])            
female_average = np.average(df.loc[df["Sex"] == "Female"]["Score"])          

# Средние для групп по свойству "Факультет"
math_average     = np.average(df.loc[df["Department"] == "Math"]["Score"])     
biology_average  = np.average(df.loc[df["Department"] == "Biology"]["Score"])  
politics_average = np.average(df.loc[df["Department"] == "Politics"]["Score"]) 

# Средние для каждых групп по всем свойствам
male_math_average       = np.average(df.loc[(df["Sex"] == "Male") & (df["Department"] == "Math")]["Score"])
female_math_average     = np.average(df.loc[(df["Sex"] == "Female") & (df["Department"] == "Math")]["Score"])
male_biology_average    = np.average(df.loc[(df["Sex"] == "Male") & (df["Department"] == "Biology")]["Score"])
female_biology_average  = np.average(df.loc[(df["Sex"] == "Female") & (df["Department"] == "Biology")]["Score"])
male_politics_average   = np.average(df.loc[(df["Sex"] == "Male") & (df["Department"] == "Politics")]["Score"])
female_politics_average = np.average(df.loc[(df["Sex"] == "Female") & (df["Department"] == "Politics")]["Score"])

# Создадим таблицу средних
avg_df = pd.DataFrame([
    ["", "Math", "Biology", "Politics", "Average"],
    ["Male", male_math_average, male_biology_average, male_politics_average, male_average],
    ["Female", female_math_average, female_biology_average, female_politics_average, female_average],
    ["Average", math_average, biology_average, politics_average, overall_average],

])

avg_df

,0,1,2,3,4
0,,Math,Biology,Politics,Average
1,Male,8.666667,5.666667,6.0,6.777778
2,Female,5.0,9.666667,6.666667,7.111111
3,Average,6.833333,7.666667,6.333333,6.944444


## Считаем суммы.
Далее посчитаем все необходимые суммы. 
Общая сумма: сумма квадратов разности каждого элемента и общего среднего
Ошибка: Сумма квадратов разности каждого элемента и среднего в его группе (по обоим свойствам)
Межгрупповая сумма по одному свойству: сумма квадратов разности среднего группы для одного свойства и общего среднего по каждому элементу
Межгрупповая сумма по обоим свойствам: Разность между общей суммой, ошибкой и межгрупповыми суммами по обоим свойствам.

In [20]:
# Общая сумма квадратов
SST = np.sum([math.pow(score - overall_average, 2) for score in df["Score"]])

# Внутригрупповая сумма квадратов
SSW = np.sum([math.pow(score - male_math_average, 2)   for score in df.loc[(df["Sex"] == "Male") & (df["Department"] == "Math")]["Score"]]) \
+ np.sum([math.pow(score - female_math_average, 2)     for score in df.loc[(df["Sex"] == "Female") & (df["Department"] == "Math")]["Score"]]) \
+ np.sum([math.pow(score - male_biology_average, 2)    for score in df.loc[(df["Sex"] == "Male") & (df["Department"] == "Biology")]["Score"]]) \
+ np.sum([math.pow(score - female_biology_average, 2)  for score in df.loc[(df["Sex"] == "Female") & (df["Department"] == "Biology")]["Score"]]) \
+ np.sum([math.pow(score - male_politics_average, 2)   for score in df.loc[(df["Sex"] == "Male") & (df["Department"] == "Politics")]["Score"]]) \
+ np.sum([math.pow(score - female_politics_average, 2) for score in df.loc[(df["Sex"] == "Female") & (df["Department"] == "Politics")]["Score"]])

# Межгрупповая сумма по свойству "Пол"
SSB_a = np.sum([math.pow(male_average - overall_average, 2) for _ in df.loc[df["Sex"] == "Male"]["Score"]]) \
+ np.sum([math.pow(female_average - overall_average, 2)     for _ in df.loc[df["Sex"] == "Female"]["Score"]])

# Межгрупповая сумма по свойству "Факультет"
SSB_b = np.sum([math.pow(math_average - overall_average, 2) for _ in df.loc[df["Department"] == "Math"]["Score"]]) \
+ np.sum([math.pow(biology_average - overall_average, 2)    for _ in df.loc[df["Department"] == "Biology"]["Score"]]) \
+ np.sum([math.pow(politics_average - overall_average, 2)   for _ in df.loc[df["Department"] == "Politics"]["Score"]])

# Межгрупповая сумма по обоим свойствам
SSB_both = SST - SSW - SSB_b - SSB_a

# Создадим таблицу сумм 
sum_df = pd.DataFrame({
    "Label" : [
        "Sum sq. for factor #1 (Sex): ", 
        "Sum sq. for factor #2 (Department)",
        "Sum sq. for both factors",
        "Error (Within): ", 
        "Total sum square: "
    ],
    "Sum Sq.": [
        SSB_a,
        SSB_b,
        SSB_both,
        SSW,
        SST
    ]
})

sum_df

,Label,Sum Sq.
0,Sum sq. for factor #1 (Sex):,0.500000
1,Sum sq. for factor #2 (Department),5.444444
2,Sum sq. for both factors,44.333333
3,Error (Within):,6.666667
4,Total sum square:,56.944444


## Степени свободы.
Теперь, получив все необходимые суммы, можем посчитать для всех видов сумм их степени свободы.

SSB_a - сумма квадратов по полу. Зная общее среднее и среднее для одного пола, можно найти среднее и для другого пола:
Количество полов (гендеров) - 1 = 2 - 1 = 1.

SSB_b - сумма квадратов по факультету. Зная общее среднее и среднее для двух факультетов, можно найти среднее и для третьего факультета:
Количество факультетов - 1 = 3 - 1 = 2.

SSB_both - сумма квадратов для обоих свойств. Просто берем произведение из количества степеней свободы по каждому свойству:
Количество степеней свободы для первого свойства * количество степеней свободы для второго свойства = 1 * 2 = 2.

SSW - ошибка (внутренняя сумма квадратов). Зная среднее в группе и m - 1 оценок, где m - количество оценок в группе, можно узнать оставшиеся оценки в каждой группе.
Количество оценок - количество групп = 18 - 6 = 12.

SST - общая сумма квадратов. Зная общее среднее и n - 1 значений оценок, можно узнать последнюю оценку.
Количество оценок - 1 = 18 - 1 = 17.

In [21]:
# Считаем степени свободы
df_SSB_a = len(["Male", "Female"]) - 1
df_SSB_b = len(["Math", "Biology", "Politics"]) - 1
df_SSB_both = df_SSB_a * df_SSB_b
df_SSW = len(df["Score"]) - len(["male_math", "male_biology", "male_politics", "female_math", "female_biology", "female_politics"])
df_SST = len(df["Score"]) - 1

# Добавляем новый столбец в таблицу
sum_df["df"] = [df_SSB_a, df_SSB_b, df_SSB_both, df_SSW, df_SST]
sum_df

,Label,Sum Sq.,df
0,Sum sq. for factor #1 (Sex):,0.500000,1
1,Sum sq. for factor #2 (Department),5.444444,2
2,Sum sq. for both factors,44.333333,2
3,Error (Within):,6.666667,12
4,Total sum square:,56.944444,17


## Средние квадраты.
Теперь каждую сумму квадратов делим на количество степеней свободы.

In [22]:
# Считаем средние квадраты
mean_SSB_a = SSB_a / df_SSB_a
mean_SSB_b = SSB_b / df_SSB_b
mean_SSB_both = SSB_both / df_SSB_both
mean_SSW = SSW / df_SSW
mean_SST = "" # Данное значение нам не понадобится

# Добавляем новый столбец
sum_df["Mean Sq."] = [mean_SSB_a, mean_SSB_b, mean_SSB_both, mean_SSW, mean_SST]
sum_df

,Label,Sum Sq.,df,Mean Sq.
0,Sum sq. for factor #1 (Sex):,0.500000,1,0.5
1,Sum sq. for factor #2 (Department),5.444444,2,2.722222
2,Sum sq. for both factors,44.333333,2,22.166667
3,Error (Within):,6.666667,12,0.555556
4,Total sum square:,56.944444,17,


## F-значения.
Считаем все значения Фишера, а именно - делим все межгрупповые суммы на внутригрупповую сумму.

In [23]:
# Считаем F-значения
f_SSB_a = mean_SSB_a / mean_SSW
f_SSB_b = mean_SSB_b / mean_SSW
f_SSB_both = mean_SSB_both / mean_SSW

# Добавляем новый столбец
sum_df["F"] = [f_SSB_a, f_SSB_b, f_SSB_both, "", ""]
sum_df

,Label,Sum Sq.,df,Mean Sq.,F
0,Sum sq. for factor #1 (Sex):,0.500000,1,0.5,0.9
1,Sum sq. for factor #2 (Department),5.444444,2,2.722222,4.9
2,Sum sq. for both factors,44.333333,2,22.166667,39.9
3,Error (Within):,6.666667,12,0.555556,
4,Total sum square:,56.944444,17,,


## p-уровень значимости
Теперь мы просто соотнесем полученные значения Фишера с F-распределением и получим p-значения.
Это я уже сделаю вручную 

In [24]:
sum_df["Pr(>F)"] = [0.361, 0.028, "0.000", "", ""]
sum_df

,Label,Sum Sq.,df,Mean Sq.,F,Pr(>F)
0,Sum sq. for factor #1 (Sex):,0.500000,1,0.5,0.9,0.361
1,Sum sq. for factor #2 (Department),5.444444,2,2.722222,4.9,0.028
2,Sum sq. for both factors,44.333333,2,22.166667,39.9,0.000
3,Error (Within):,6.666667,12,0.555556,,
4,Total sum square:,56.944444,17,,,


## Интерпретация.
Как видно из таблицы выше, значениями p-value, которые больше 0.05, обладают только сумма квадратов по свойству "Факультет" и сумма квадратов по обоим свойствам. Этот вывод можно сформулировать следующим образом: *"Только пол статитстически значимо не влияет на успеваемость студентов. С другой стороны, факультет, в котором учится студент, статистически значимо влияет на успеваемость студентов. Однако в комбинации, и пол, и факультет еще более статистически значимо влияют на успеваемость студента."* 

## Самопроверка.
Автоматическое применение Two-Way ANOVA с помощью модуля *statsmodels*:

In [25]:
model = ols('Score ~ C(Sex) + C(Department) + C(Sex):C(Department)', data=df).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Sex),0.500000,1.0,0.9,0.361497
C(Department),5.444444,2.0,4.9,0.027819
C(Sex):C(Department),44.333333,2.0,39.9,0.000005
Residual,6.666667,12.0,NaN,NaN


В данном выводе *Residual* ознакает ошибку (внутригрупповую сумму квадратов), *C(Sex)* И *C(Department)* - сумма квадратов по одному свойству. А *C(Sex):C(Department)* - сумма квадратов по обоим свойствам. 